In [7]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm_notebook as tqdm


In [10]:
# modified from https://www.kaggle.com/tanlikesmath/diabetic-retinopathy-resnet50-binary-cropped
import cv2
def resize_images(location, name, extension, resize_location, desired_size = 1024):
    img = cv2.imread(f"{location}/{name}.{extension}")
    
    img = cv2.copyMakeBorder(img,10,10,10,10,cv2.BORDER_CONSTANT,value=[0,0,0])
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret,gray = cv2.threshold(gray,10,255,cv2.THRESH_BINARY)
    
    contours,hierarchy = cv2.findContours(gray,
                                          cv2.RETR_EXTERNAL,
                                          cv2.CHAIN_APPROX_SIMPLE)
    contours = max(contours, key=cv2.contourArea)
    x,y,w,h = cv2.boundingRect(contours)

    if w>200 and h>200:
        new_img = img[y:y+h,x:x+w]
        height, width, _= new_img.shape


        new_img = cv2.resize(new_img,(desired_size,desired_size), 
                                 interpolation = cv2.INTER_CUBIC)
            
        cv2.imwrite(f'{resize_location}/{name}.jpg', new_img)
    else:
        print(f'No bounding for {name}')
        img = cv2.resize(img,(desired_size,desired_size), 
                                 interpolation = cv2.INTER_CUBIC)
        cv2.imwrite(f'{resize_location}/{name}.jpg', img)

In [11]:
path = Path("input")
train_19_df = pd.read_csv(path/'trainLabels19.csv')
test_19_df = pd.read_csv(path/'testImages19.csv')
train_15_df = pd.read_csv(path/'trainLabels15.csv')
test_15_df = pd.read_csv(path/'testLabels15.csv')

train_19_df.head() 

,id_code,diagnosis
0,000c1434d8d7,2
1,001639a390f0,4
2,0024cdab0c1e,1
3,002c21358ce6,0
4,005b95c28852,0


In [12]:
test_15_df.rename(index=str, columns={"image": "id_code", "level":"diagnosis"}, inplace=True)
train_15_df.rename(index=str, columns={"image": "id_code", "level":"diagnosis"}, inplace=True)

data_df = train_19_df.append([test_15_df.drop('Usage', axis=1), train_15_df], ignore_index = True)

In [13]:
data_df.shape

(92364, 2)

In [14]:
for index, row in tqdm(data_df.iterrows(), total=data_df.shape[0]):
    resize_images('input/resized', row['id_code'], 'jpg', 'input/resized_256', desired_size = 256)

No bounding for 3517_left
No bounding for 16519_left
No bounding for 19150_right
No bounding for 25313_right
No bounding for 27096_right
No bounding for 28544_left
No bounding for 35762_left
No bounding for 38549_left
No bounding for 1986_left
No bounding for 3829_left
No bounding for 32253_right
No bounding for 34689_left
No bounding for 43457_left
